## 3. Uncertainty Quantification

In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import datetime
import h5py
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import sys

from cv2 import resize
from datetime import datetime
from gc import collect
from os import cpu_count
from sklearn.model_selection import train_test_split
from time import sleep
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from tqdm import tqdm

sys.path.append(f"{os.getcwd()}/working ViT/")
sys.path.append(f"{os.getcwd()}/mltfm/")
from VisionTransformer_working import VisionTransformer as vit_old

2023-01-02 14:16:53.724953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 14:16:53.893290: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-02 14:16:54.487632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/alexrichard/miniconda3/lib/
2023-01-02 14:16:54.487721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.s

In [2]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
cudnn.benchmark = True

In [3]:
collect()
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Running on device: {device}")

Running on device: cpu


### MC Dropout

Load CNN and ViT model.

In [4]:
vit = vit_old(dspl_size=104, 
              patch_size=8, 
              embed_dim=128,
              depth=12,
              n_heads=8,
              mlp_ratio=4.,
              p=0.,
              attn_p=0.,
              drop_path=0.).double()

In [5]:
path_to_pth = 'working ViT/logs_and_weights/ViT-2022-Dec-10 23:18:41/ViT-2022-Dec-10 23:18:41_best_val_loss_2.365828e-06.pth'

In [6]:
if torch.cuda.is_available():
    vit.load_state_dict(torch.load(path_to_pth)['best_model_weights'], strict=True)
else:
    vit.load_state_dict(torch.load(path_to_pth, map_location=torch.device('cpu'))['best_model_weights'], strict=True)        

In [7]:
os.getcwd()

'/home/alexrichard/PycharmProjects/UQ_DL-TFM/ViT-TFM'

In [8]:
cnn = keras.models.load_model('../mltfm/models/model_noise_1e-4.h5')

2023-01-02 14:17:23.016281: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-02 14:17:23.021269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-02 14:17:23.021667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-02 14:17:23.022577: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

2023-01-02 14:17:23.783117: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 2.2K (2304 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-01-02 14:17:23.783784: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 2.2K (2304 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-01-02 14:17:23.784445: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 2.2K (2304 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-01-02 14:17:23.785110: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 2.2K (2304 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-01-02 14:17:23.785774: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 2.2K (2304 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-01-02 14:17:23.786450: I tensorflow/stream_executor/cuda/cuda_driver.cc:733] failed to allocate 2.2K (2304 bytes) from device: CUDA_ERR


KeyboardInterrupt



Enable dropout during inference in the CNN.

In [ ]:
config = cnn_weights.get_config()

for i, layer in enumerate(config['layers']):
    if layer['class_name'] == 'Dropout':
        layer['inbound_nodes'][0][0][-1]['training'] = True
        
cnn_ = keras.Model.from_config(config)
cnn_.load_weights('mltfm/models/model_noise_1e-4.h5')

For both the CNN and ViT, compute `n_forward_passes` for each of the 25 test samples. Note that the CNN contains 6 dropout layers (p=0.1) and the ViT contains 12 dropout layers in the MLP module of each encoder block (p=0.1).

In [ ]:
from main_dist import inference_with_dropout

n_forward_passes = 10

predictions_vit = torch.zeros(25, 10, 2, 104, 104)
predictions_cnn = torch.zeros(25, 10, 2, 104, 104)
for i in range(0, len(X_test)):
    for j in range(0, n_forward_passes):
        predictions_vit[i, j, :, :, :] = inference_with_dropout(vit_weights, X_test[i][np.newaxis, ...], return_attn_scores=False)
        predictions_cnn[i, j, :, :, :] = torch.tensor(np.moveaxis(np.array((cnn_(dspl[i][np.newaxis, ...]))), 3, 1))

In [ ]:
Calculate loss per prediction and loss of averaged prediction.

In [ ]:
averaged_pred_vit = 1/n_forward_passes * predictions_vit.sum(dim=1)
averaged_pred_cnn = 1/n_forward_passes * predictions_cnn.sum(dim=1)

losses_vit = torch.zeros(25, 10)
losses_cnn = torch.zeros(25, 10)
for i in range(0, len(X_test)):
    for j in range(0, n_forward_passes):
        losses_vit[i, j] = torch.sum(loss(predictions_vit[i, j], Y_test[i]))
        losses_cnn[i, j] = torch.sum(loss(predictions_cnn[i, j], Y_test[i]))

In [ ]:
loss_mean_per_sample_vit = torch.mean(losses_vit, axis=1)
loss_mean_per_sample_cnn = torch.mean(losses_cnn, axis=1)

In [ ]:
loss_variance_per_sample_vit = torch.var(losses_vit, axis=1)
loss_variance_per_sample_cnn = torch.var(losses_cnn, axis=1)

In [ ]:
avg_test_rmse_vit = torch.mean(loss_mean_per_sample_vit)
avg_test_rmse_cnn = torch.mean(loss_mean_per_sample_cnn)

In [ ]:
avg_test_rmse_cnn

In [ ]:
avg_test_rmse_vit

Compute empirical variances for each predicted value.

In [ ]:
variances_vit = np.var(predictions_vit.detach().numpy(), axis=1)
variances_cnn = np.var(predictions_cnn.detach().numpy(), axis=1)

Plot predicted variances for one in-distribution test sample.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(2,2, figsize=(9, 8))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)

im = axs[0, 0].imshow(variances_vit[0, 0, :, :], cmap='jet', interpolation = 'nearest')
axs[0, 0].set_title('test_sample_00: Variance of predictions in x-plane (ViT)', {'fontsize': 10})
divider = make_axes_locatable(axs[0, 0])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

im = axs[0, 1].imshow(variances_vit[0, 1, :, :], cmap='jet', interpolation = 'nearest')
axs[0, 1].set_title('test_sample_00: Variance of predictions in y-plane (ViT)', {'fontsize': 10})
divider = make_axes_locatable(axs[0, 1])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

im = axs[1, 0].imshow(variances_cnn[0, 0, :, :], cmap='jet', interpolation = 'nearest')
axs[1, 0].set_title('test_sample_00: Variance of predictions in x-plane (CNN)', {'fontsize': 10})
divider = make_axes_locatable(axs[1, 0])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

im = axs[1, 1].imshow(variances_cnn[0, 1, :, :], cmap='jet', interpolation = 'nearest')
axs[1, 1].set_title('test_sample_00: Variance of predictions in y-plane (CNN)', {'fontsize': 10})
divider = make_axes_locatable(axs[1, 1])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

Corrupt test samples with Gaussian noise (i.e. create out-of-distribution samples).

In [ ]:
sigma = 1e-3
cov = torch.tensor([[sigma**2,0],[0,sigma**2]])
noise = np.random.multivariate_normal(np.array([0,0]),cov, (len(X_test),104,104))
noisy_test = dspl + noise

In [ ]:
dspl_noisy_ = np.moveaxis(np.array(noisy_test),3 ,1)
X_test_noisy = torch.from_numpy(dspl_noisy_).double()

In [ ]:
noisy_predictions_vit = torch.zeros(1, 10, 2, 104, 104)
noisy_predictions_cnn = torch.zeros(1, 10, 2, 104, 104)
        
noisy_predictions_vit[0, :, :, :, :] = inference_with_dropout(vit_weights, X_test_noisy[0][np.newaxis, ...], return_attn_scores=False)
noisy_predictions_cnn[0, :, :, :, :] = torch.tensor(np.moveaxis(np.array((cnn_(noisy_test[0][np.newaxis, ...]))), 3, 1))

In [ ]:
averaged_pred_vit = 1/n_forward_passes * noisy_predictions_vit.sum(dim=1)
averaged_pred_cnn = 1/n_forward_passes * noisy_predictions_cnn.sum(dim=1)

losses_noisy_vit = torch.zeros(1, 10)
losses_noisy_cnn = torch.zeros(1, 10)
for i in range(0, len(noisy_predictions_vit)):
    for j in range(0, n_forward_passes):
        losses_noisy_vit[i, j] = torch.sum(loss(noisy_predictions_vit[i, j], Y_test[i]))
        losses_noisy_cnn[i, j] = torch.sum(loss(noisy_predictions_cnn[i, j], Y_test[i]))

In [ ]:
loss_mean_per_noisy_sample_vit = torch.mean(losses_vit, axis=1)
loss_mean_per_noisy_sample_cnn = torch.mean(losses_cnn, axis=1)

In [ ]:
variances_noisy_vit = np.var(noisy_predictions_vit.detach().numpy(), axis=1)
variances_noisy_cnn = np.var(noisy_predictions_cnn.detach().numpy(), axis=1)

In [ ]:
loss_of_averaged_vit_pred = torch.sum(loss(averaged_pred_vit, Y_test[0]))
loss_of_averaged_cnn_pred = torch.sum(loss(averaged_pred_cnn, Y_test[0]))

In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(2,2, figsize=(9, 9))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)
axs[0, 0].quiver(dspl[0,:,:,0], dspl[0,:,:,1], scale=1)
axs[0, 0].set_title('Input (test_sample_00)', {'fontsize': 11})

axs[0, 1].quiver(trac[0,:,:,0], trac[0,:,:,1], scale=10)
axs[0, 1].set_title('Ground truth (test_sample_00)', {'fontsize': 11})

axs[1, 0].quiver(averaged_pred_vit[0,0,:,:].detach().numpy(), averaged_pred_vit[0,1,:,:].detach().numpy(), scale=10)
axs[1, 0].set_title(f'ViT prediction (loss: {loss_of_averaged_vit_pred})', {'fontsize': 11})

axs[1, 1].quiver(averaged_pred_cnn[0,0,:,:], averaged_pred_cnn[0,1,:,:], scale=10)
axs[1, 1].set_title(f'CNN prediction (loss: {loss_of_averaged_cnn_pred})', {'fontsize': 11})

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(9, 8))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)

im = axs[0, 0].imshow(variances_noisy_vit[0, 0, :, :], cmap='jet', interpolation = 'nearest')
axs[0, 0].set_title('test_sample_00: Variance of predictions in x-plane (ViT)', {'fontsize': 10})
divider = make_axes_locatable(axs[0, 0])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

im = axs[0, 1].imshow(variances_noisy_vit[0, 1, :, :], cmap='jet', interpolation = 'nearest')
axs[0, 1].set_title('test_sample_00: Variance of predictions in y-plane (ViT)', {'fontsize': 10})
divider = make_axes_locatable(axs[0, 1])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

im = axs[1, 0].imshow(variances_noisy_cnn[0, 0, :, :], cmap='jet', interpolation = 'nearest')
axs[1, 0].set_title('test_sample_00: Variance of predictions in x-plane (CNN)', {'fontsize': 10})
divider = make_axes_locatable(axs[1, 0])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

im = axs[1, 1].imshow(variances_noisy_cnn[0, 1, :, :], cmap='jet', interpolation = 'nearest')
axs[1, 1].set_title('test_sample_00: Variance of predictions in y-plane (CNN)', {'fontsize': 10})
divider = make_axes_locatable(axs[1, 1])
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

In [ ]:
Input an all-zero displacement field.

In [ ]:
vit_weights.eval()
all_zeros = torch.zeros(2,104,104).double()

In [ ]:
pred_vit = vit_weights(all_zeros[np.newaxis, ...])

In [ ]:
loss = torch.sum(loss(pred_vit[0], all_zeros))
loss

In [ ]:
get_ipython().run_line_magic('matplotlib', 'notebook')

# Plots
fig, axs = plt.subplots(1,1, figsize=(6, 6))
fig.tight_layout(pad=3, w_pad=3, h_pad=3)
axs.quiver(pred_vit[0][0,:,:].detach().numpy(), pred_vit[0][1,:,:].detach().numpy(), scale=1)
axs.set_title('Input (test_sample_00)', {'fontsize': 11})

In [ ]:
"""
mean = variances_vit[:,0,0,0].detach().numpy().mean()
var = variances_vit[:,0,0,0].detach().numpy().var()
gt = test_gt_00[0,0,0,0]


fig, ax = plt.subplots(figsize=(7, 7))
textstr = '\n'.join((
    r'$\mathrm{mean}=%.1e$' % (mean),
    r'$\mathrm{var}=%.1e$' % (var)))
    
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
ax.text(0.03, 0.97, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

counts, bins = np.histogram(stacked_predictions_vit[:,0,0,0].detach().numpy(), bins=50)
max_counts = np.max(counts)
ax.stairs(counts, bins)
ax.hist(bins[:-1], bins, weights=counts, color='dodgerblue', ec='black')
ax.axvline(mean, color='k', linestyle='dashed', linewidth=1)
ax.axvline(mean, color='black', linestyle='dashed', linewidth=1)
ax.axvline(gt, color='mediumseagreen', linestyle='dashed', linewidth=1.5, label='ground truth')
ax.text(mean*1.1, 0.85*max_counts, 'mean', rotation=90, fontsize=7)
ax.text(gt*1.12, 0.85*max_counts, 'ground truth', rotation=90, fontsize=7)
ax.set_ylabel('Counts')
ax.set_xlabel('Predicted values');
"""

### Distance in latent space

"What the neural net is essentially doing is stripping away details from the features that are not helpful and amplifying the most important ones, and by the time we get to the latent space, it’s basically the way the model sees the molecule to make the prediction."

In [ ]:
dspl = h5py.File('data/displacements_25.h5')["data"]
trac = h5py.File('data/tractions_25.h5')["data"]

In [ ]:
dspl_ae = np.array(dspl[0:110, :, :, :])
dspl_vit = np.moveaxis(np.array(dspl_ae),3 ,1)

X_train_ae = torch.from_numpy(dspl_ae).double()
X_test_vit = torch.from_numpy(dspl_vit).double()

In [ ]:
class LatentRepresentationExtractor(nn.Module):
    def __init__(self, patch_embed, pos_embed, blocks):
        super().__init__()
        self.patch_embed = patch_embed
        self.pos_embed = pos_embed
        self.blocks = blocks
        
    def forward(self, x):
        x = self.patch_embed(x)
        x = x + self.pos_embed  # (n_samples, n_patches, embed_dim)

        for block in self.blocks:
            x, _ = block(x)
        
        return x

In [ ]:
representation_extractor = LatentRepresentationExtractor(vit_weights.patch_embed, vit_weights.pos_embed, vit_weights.blocks)

In [ ]:
representations = representation_extractor(X_test_vit)

In [ ]:
representations = representations.detach()

In [ ]:
class DeepAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(10 * 169 , 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 8)
        )
        
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(8, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 10 * 169),
        )

    def forward(self, x):
        x = x.reshape(-1, 10 * 169)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = DeepAutoencoder().to(device).double()

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [ ]:
X_train = representations[0:100, :, :].double()
X_val = representations[100:, :, :].double()

train_set = TensorDataset(X_train, X_train)
val_set = TensorDataset(X_val, X_val)

dataloader_kwargs = {'batch_size': 10,
                     'num_workers': 10,
                     'pin_memory': False,
                     'shuffle': True}
dataloaders = {'train': DataLoader(train_set, **dataloader_kwargs),
               'val': DataLoader(val_set, **dataloader_kwargs)}

In [ ]:
def run_epoch(model, loss_fn, dataloader, device, epoch, optimizer, train, visualize_attn=False):
    # Set model to training mode
    if train:
        model.train()
    else:
        model.eval()

    epoch_loss = 0.0

    with tqdm(dataloader, unit="batch") as tepoch:
        # Iterate over data
        for xb, yb in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            xb, yb = xb.to(device), yb.to(device)
            yb = yb.reshape(-1, 10 * 169)

            # zero the parameters
            if train:
                optimizer.zero_grad(set_to_none=True)

            # forward
            if visualize_attn:
                pred, attn_scores = model(xb)  # attn_scores: [attn_1, ..., attn_depth]
            else:
                pred = model(xb)
            if not train and visualize_attn:
                with torch.set_grad_enabled(not train):
                    attn_mat = torch.stack(attn_scores) # attn_mat.shape==(depth, samples, n_heads, n_patches**0.5, n_patches**0.5)
                    attn_mat = attn_mat.squeeze(1) # shape remains unchanged unless samples==1, then: (depth, n_heads, n_patches**0.5, n_patches**0.5)
                    attn_mat = torch.mean(attn_mat, dim=1) # if samples==1: attn_mat.shape==(depth, n_patches**0.5,n_patches**0.5)

            with torch.set_grad_enabled(train):
                loss = loss_fn(pred, yb)

                # backward + optimize if in training phase
                if train:
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)
                    optimizer.step()

            # statistics
            epoch_loss += loss.item()

        epoch_loss /= len(dataloader.dataset)
        tepoch.set_postfix(loss=epoch_loss)
        sleep(0.01)
    return epoch_loss

In [ ]:
def fit(model, loss_fn, dataloaders, optimizer, device, writer, NAME, max_epochs, patience):
    best_val_loss = np.inf
    best_epoch = -1
    best_model_weights = {}
    optimizer_state_dict_in_best_epoch = {}
    model.to(device)
    example_input, example_value = next(iter(dataloaders['train']))
    writer.add_graph(model, example_input.to(device))

    for epoch in range(1, max_epochs + 1):
        train_loss = run_epoch(model, loss_fn, dataloaders['train'], device, epoch, optimizer, train=True)
        val_loss = run_epoch(model, loss_fn, dataloaders['val'], device, epoch, optimizer=None, train=False)
        print(
            f"Epoch {epoch}/{max_epochs}, train_loss: {train_loss}")

        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('val_loss', val_loss, epoch)

        # Save best weights
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_model_weights = copy.deepcopy(model.state_dict())
            optimizer_state_dict_in_best_epoch = copy.deepcopy(optimizer.state_dict())

        # Early stopping
        print(
            f"best_val_loss: {np.round(best_val_loss, 6)}, "
            f"epoch: {epoch}, best_epoch: {best_epoch}, "
            f"current_patience: {patience - (epoch - best_epoch)}")
        if epoch - best_epoch >= patience:
            break

    torch.save({
        'best_epoch': best_epoch,
        'best_model_weights': best_model_weights,
        'optimizer_state_dict_in_best_epoch': optimizer_state_dict_in_best_epoch,
        'best_val_loss': best_val_loss
    },
        f'logs_and_weights/{NAME}/{NAME}_best_val_loss_{np.round(best_val_loss, 6)}.pth')



In [ ]:
NAME = "AE-{:%Y-%b-%d %H:%M:%S}".format(datetime.now())
writer = SummaryWriter(log_dir=f'logs_and_weights/{NAME}')
fit(model, criterion, dataloaders, optimizer, device, writer, NAME, 100, 20)
writer.close()

In [ ]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [ ]:
new_point = dspl[111, :, :, :]

In [ ]:
new_point = np.moveaxis(np.array(new_point[np.newaxis]),3 ,1)

In [ ]:
new_point = torch.from_numpy(new_point).double()

In [ ]:
new_point_rep = representation_extractor(new_point)

### [Gaussian process layers](https://arxiv.org/pdf/2106.00638.pdf)

__Goal__: Use variance of GP as distance measure between new input and already observed training data.

__Idea__: 
  1. Combination of Deep Neural Networks (DNNs) and Gaussian Processes (GPs)
  2. Scalable Variational Gaussian Processes (SVGPs): Complexity of O(m³) with m as number of inducing points
  3. Use the outputs of a neural network as the input to an uncertainty aware GP
  4. Given a new input sample, the GP model provides a predictive distribution over (univariate!) target variable

__Algorithm__: 
  1. Train backbone (e.g. CNN, Transformer) on some problem
  2. Generate inducing points by feeding images into the model and embed them
  3. Use these embedding points as inducing points for SVGP
  4. Maximize SVGP parameters (e.g. scaling parameter of RBF kernel) by maximizing ELBO

__Problem__: For every target variable, one SVGP has to be trained. Furthermore, SVGPs assume independence between the target variables.